<a href="https://www.kaggle.com/code/samithsachidanandan/generating-movie-dialogs-with-rnn-with-python?scriptVersionId=266769910" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import random 
import numpy as np 
import pandas as pd
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import LSTM, Dense, Activation 
from tensorflow.keras.optimizers import RMSprop

2025-10-09 08:47:19.242318: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759999639.471089      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759999639.537150      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv("/kaggle/input/movie-dialog-corpus/movie_lines.tsv", sep='\t', header=None, quoting=3, on_bad_lines='skip')

In [3]:
text_data = df.iloc[:, -1].dropna().astype(str)

In [4]:
text = " ".join(text_data).lower()

In [5]:
text = text[300000:800000]

In [6]:
characters = sorted(set(text))

In [7]:
char_to_index = dict((c,i) for i, c in enumerate(characters))
index_to_char = dict((i,c) for i, c in enumerate(characters))

In [8]:
SEQ_LENGTH = 40
STEP_SIZE = 3

In [9]:
sentences = []
next_characters = []


In [10]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i+SEQ_LENGTH])
    next_characters.append(text[i+SEQ_LENGTH])


In [11]:
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=bool)
y = np.zeros((len(sentences), len(characters)), dtype=bool)

In [12]:
for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        x[i, t, char_to_index[character]] = 1
    y[i, char_to_index[next_characters[i]]] = 1
    

In [13]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

optimizer = RMSprop(learning_rate=0.01)

model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.fit(x, y, batch_size=256, epochs=4)

model.save('textgenerator.keras')

model.summary()

2025-10-09 08:47:38.557584: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 63s 94ms/step - loss: 2.4966
Epoch 2/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 61s 94ms/step - loss: 1.8167
Epoch 3/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 61s 94ms/step - loss: 1.6584
Epoch 4/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 61s 94ms/step - loss: 1.5773


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        95,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 58)             │         7,482 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 58)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 206,454 (806.46 KB)

 Trainable params: 103,226 (403.23 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 103,228 (403.24 KB)

In [14]:
model = tf.keras.models.load_model('textgenerator.keras')

In [15]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds =np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



In [16]:
def generate_text(length, temperature): 
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence 
    for i in range(length): 
        x = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, character in enumerate(sentence):
            x[0, t, char_to_index[character]] = 1

        predictions = model.predict(x, verbose=0)[0]
        next_index = sample(predictions, temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated 

In [17]:
print(generate_text(300,0.2))

ow! i mean you're a delicious delightful and i can't be a confuld i don't know i thought the compan and i can't say the really make the compors and do you know i don't know i don't know i don't know i think i must want to the party to the traids the said i was the party something the say i was not the saying in the compors in the compors 


In [18]:
print(generate_text(300,0.4))

nies baby bells picture libraries museumanially with some services. the beallers. i thought i come better says and i say that a bash a draghe of the coplack to the moon. i was the party the pard to the really the parded the taple to the party this is a lot of the compors and the fambons don't what is something to the say of the partwer --


In [19]:
print(generate_text(300,0.6))

great title for a song! i'm gonna write some wride a the rad.  gonna me the nown i could do the really to kid the conforsting. what's the end the other stayzard to see you is the must he work you can think you would let was a please is for it have that i say lot is a lot my blay. what is it is look -- why were i got a good our party the m


In [20]:
print(generate_text(300,0.8))

eries and no evil puppet masters behind secrice dony? they readsher. yes! show could patsinker? i all the work of the reaids come a little only furny where they're and movark. monshhe! i'm morng? it's not a buy tome been becauss.  we all the but you don't be what is the an insil a firat? and diskeass alborgicrined the compan coming furnaa


In [21]:
print(generate_text(300,1.0))

e. he's too important james.  they'll see you buy be arshaled... tell the copast. thank in.. he body. newing been wheep shere you don't don't mave you away. dablat! crlass? ser you're work but okay? at the nistryway i waong to ale hore the omorning you. just annwowntalve you. you're prodrately virnems? oh i can.  where's you have bro pear
